# Recommendation Systems – Frequent PaƩern Mining

You are given the market basket dataset (market-basket.csv).
- Use the FP-growth algorithm in Apache Spark to find the most frequent items. Save
your result into an .xlsx files.
- Do the experiment with different minSupport and minConfidence values. Give the
conclusion at the end of experiment.

In [4]:
!pip install gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [17]:
import gdown
import os

url = "https://drive.google.com/file/d/19OMIjRgshCsIZvUbpvrdbAKl6O2sRtV6/view?usp=share_link"
output_path = "/content/sample_data/market-basket.csv"
gdown.download(url, quiet=False,fuzzy=True)

Downloading...
From: https://drive.google.com/uc?id=19OMIjRgshCsIZvUbpvrdbAKl6O2sRtV6
To: /content/market-basket.csv
100%|██████████| 41.2M/41.2M [00:00<00:00, 210MB/s]


'market-basket.csv'

In [19]:
import pandas as pd

In [36]:
data = pd.read_csv('market-basket.csv', on_bad_lines='skip' , delimiter=";")
data.head()

<ipython-input-36-6f7c58680fad>:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('market-basket.csv', on_bad_lines='skip' , delimiter=";")


,BillNo,Itemname,Quantity,Date,Price,CustomerID,Country
0,536365,WHITE HANGING HEART T-LIGHT HOLDER,6,01.12.2010 08:26,"2,55",17850.0,United Kingdom
1,536365,WHITE METAL LANTERN,6,01.12.2010 08:26,"3,39",17850.0,United Kingdom
2,536365,CREAM CUPID HEARTS COAT HANGER,8,01.12.2010 08:26,"2,75",17850.0,United Kingdom
3,536365,KNITTED UNION FLAG HOT WATER BOTTLE,6,01.12.2010 08:26,"3,39",17850.0,United Kingdom
4,536365,RED WOOLLY HOTTIE WHITE HEART.,6,01.12.2010 08:26,"3,39",17850.0,United Kingdom


In [23]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=4954971bd02890aefdc98b62ab580cd42504df2d7e467c43d5430fff6b1d45cb
  Stored in directory: /root/.cache/pip/wheels/9f/34/a4/159aa12d0a510d5ff7c8f0220abbea42e5d81ecf588c4fd884
Successfully built pyspark


In [24]:
# Create the Spark Session
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Streaming Process Files") \
    .config("spark.streaming.stopGracefullyOnShutdown", True) \
    .master("local[*]") \
    .getOrCreate()

spark

In [65]:
from google.colab import files
from pyspark.sql import functions as F
from pyspark.ml.fpm import FPGrowth
import pandas

sparkdata = spark.createDataFrame(data)

In [66]:
from pyspark.ml.fpm import FPGrowth
from pyspark.sql.functions import collect_set
from mlxtend.preprocessing import TransactionEncoder

In [67]:
# basketdata = sparkdata.dropDuplicates(['Billno', 'Itemname']).sort('Billno')
basketdata = sparkdata.groupBy('Billno').agg(collect_set('Itemname').alias('items'))
basketdata.show(5, truncate=False)
basketdata.show()

+------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Billno|items                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

In [68]:
fpGrowth = FPGrowth(minSupport=0.006, minConfidence=0.006)
model = fpGrowth.fit(basketdata)
items = model.freqItemsets

In [ ]:
# transform examines the input items against all the association rules and summarize the
# consequents as prediction
model.transform(basketdata).show()
transformed = model.transform(basketdata)

In [ ]:
# Convert the Spark DataFrame back to a Pandas DataFrame using Arrow
result_pdf = items.select("*").toPandas()
result_pdf.head()

In [ ]:
result_pdf.to_excel('result_pdfItemsFreq.xlsx')